# PcGrad Implementation using Python
Pc-Grad is an optimized gradient descent algorithm which is mainly used for multitask-training.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pdb
import numpy as np
import copy
import random

from torch.optim.adamw import adamw

In [ ]:
class PCGrad():
    def __init__(self, optimizer, reduction='mean'):
        self.optimizer = optimizer
        return

    def zero_grad(self):
        return self.optimizer.zero_grad()(set_to_none=True)

    def pack_gradient(self, objectives):
        grads, shapes, has_grads = [], [], []
        for objective in objectives:
            self.optimizer.zero_grad(set_to_none=True)

Use a seed for replication and assign random values for input and expected results for testing. Further specifiy a fixed
tensor assuming it's a prediction.

In [9]:
torch.manual_seed(4)
input, expectations = torch.randn(2, 3), torch.randn(2, 4)
predictions = torch.tensor([[-1.6053,  0.2325,  2.2399],
                           [ 0.8473,  1.2006, -0.4016]])
print(x,y)


tensor([[-1.6053,  0.2325,  2.2399],
        [ 0.8473,  1.2006, -0.4016]]) tensor([[-1.4260,  0.9039,  0.8557,  0.6889],
        [ 0.8850,  1.7706, -0.0809,  0.0513]])


Compute the losses of the current predictions from the model. First use the derivation of the MSE functions
and then apply the loss functions on the input and predictions done by the model, this is
our starting point for PCGrad.

In [10]:
loss1_fn, loss2_fn = nn.L1Loss(), nn.MSELoss()
print(loss1_fn(input, predictions), loss2_fn(input, predictions))

tensor(2.8007e-05) tensor(1.0211e-09)


Create a random tensor which is considered as our parameters. Then initialize an optimizer.

In [ ]:
parameter = torch.randn(2, 3)
optimizer = torch.optim.Adam(parameter)

Now we compute the projected conflicted gradient or the gradients for the give loss objectives.

In [ ]:
def pc_backward(objectives):
    grads, shapes, has_grads = pc_package_grad(objectives)

In [ ]:
def pc_package_grad(objectives):
    grads, shapes, has_grads = [], [], []
    for objective in objectives:
        optimizer.zero_grad(set_to_none=True)
        objective.backward(retain_grad=True)
        grad, shape, has_grad = pc_retrieve_grad()

The function will get the gradient of the parameters form the given model/network with specific objective.
It will differentiate between gradients that actually present in the given parameter tensor or not.


In [ ]:
def pc_retrieve_grad(self):
    grad, shape, has_grad = [], [], []
    for group in optimizer.param_groups:
        for param in group['params']:
            if param.grad is None:
                shape.append(param.shape)
                grad.append(torch.zeros_like(param).to(param.device))
                has_grad.append(torch.zeros_like(param).to(param.device))
                continue
            shape.append(param.grad.shape)
            grad.append(param.grad.clone())
            has_grad.append(torch.ones_like(param).to(param.device))
    return grad, shape, has_grad
